In [3]:
pip install face_recognition scikit-learn xgboost numpy matplotlib



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import shutil
from PIL import Image

# 1. First verify current structure
print("Current files in known_faces:")
for item in os.listdir("known_faces"):
    item_path = os.path.join("known_faces", item)
    if os.path.isdir(item_path):
        print(f"📁 {item}/")
        for subitem in os.listdir(item_path):
            print(f"  ├─ {subitem}")
    else:
        print(f"📄 {item}")

# 2. Reorganization code - ADJUSTED FOR YOUR ACTUAL STRUCTURE
try:
    # Convert WebP to JPG for person2
    webp_path = "known_faces/person2/mesh.webp"
    if os.path.exists(webp_path):
        img = Image.open(webp_path).convert("RGB")
        jpg_path = "known_faces/person2/person2_1.jpg"
        img.save(jpg_path)
        os.remove(webp_path)
        print(f"\n✅ Converted {webp_path} to {jpg_path}")
    
    # Rename person1's image
    old_path = "known_faces/person1/myface.jpg"
    new_path = "known_faces/person1/person1_1.jpg"
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
        print(f"✅ Renamed {old_path} to {new_path}")
    
    # No ronaldo.jpg exists based on your screenshots
    # If you have it, please specify its exact location
    
    print("\nReorganization complete! New structure:")
    for person in os.listdir("known_faces"):
        if os.path.isdir(os.path.join("known_faces", person)):
            print(f"\n👤 {person}:")
            for img in os.listdir(os.path.join("known_faces", person)):
                print(f"  - {img}")

except Exception as e:
    print(f"\n❌ Error occurred: {str(e)}")
    print("Please check:")
    print("1. File paths are correct")
    print("2. Files actually exist in the shown locations")
    print("3. You have proper permissions")

Current files in known_faces:
📁 .ipynb_checkpoints/
📁 Amar/
  ├─ .ipynb_checkpoints
  ├─ Amar_1.jpg
  ├─ person1_2.jpg
📁 Arpita_singh/
  ├─ .ipynb_checkpoints
  ├─ Arpita_singh.png
📁 person1/
  ├─ .ipynb_checkpoints
  ├─ sample.jpg
📁 person2/
  ├─ .ipynb_checkpoints
  ├─ person2_1.jpg
  ├─ person2_1749916628.jpg
📁 ronaldo/
  ├─ .ipynb_checkpoints
  ├─ ronaldo_1.jpg

Reorganization complete! New structure:

👤 .ipynb_checkpoints:

👤 Amar:
  - .ipynb_checkpoints
  - Amar_1.jpg
  - person1_2.jpg

👤 Arpita_singh:
  - .ipynb_checkpoints
  - Arpita_singh.png

👤 person1:
  - .ipynb_checkpoints
  - sample.jpg

👤 person2:
  - .ipynb_checkpoints
  - person2_1.jpg
  - person2_1749916628.jpg

👤 ronaldo:
  - .ipynb_checkpoints
  - ronaldo_1.jpg


In [7]:
import os
import shutil

# Remove hidden checkpoints folders
for root, dirs, files in os.walk("known_faces"):
    if '.ipynb_checkpoints' in dirs:
        shutil.rmtree(os.path.join(root, '.ipynb_checkpoints'))
        print(f"Removed {os.path.join(root, '.ipynb_checkpoints')}")

# Move loose files to correct folders
if os.path.exists("known_faces/myyface.jpg"):
    shutil.move("known_faces/myyface.jpg", "known_faces/person1/myyface.jpg")
if os.path.exists("known_faces/mesi.webp"):
    shutil.move("known_faces/mesi.webp", "known_faces/person2/mesi.webp")

print("Cleanup complete!")

Cleanup complete!


In [5]:
import cv2
import os
import face_recognition
import numpy as np
from datetime import datetime

# Configuration
TARGET_IMAGES = 5
RECOG_THRESH = 0.45
FRAME_SCALE = 0.5
PROCESS_EVERY = 2

def setup_dirs():
    os.makedirs("known_faces/Amar", exist_ok=True)
    os.makedirs("known_faces/Arpita_singh", exist_ok=True)

def load_encodings_fast():
    encodings = []
    names = []
    
    for person in ["Amar", "Arpita_singh"]:
        person_dir = f"known_faces/{person}"
        if not os.path.exists(person_dir):
            continue
            
        for img_file in os.listdir(person_dir)[:5]:
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
                
            img_path = f"{person_dir}/{img_file}"
            image = face_recognition.load_image_file(img_path)
            face_locs = face_recognition.face_locations(image)
            
            if face_locs:
                enc = face_recognition.face_encodings(image, known_face_locations=face_locs, num_jitters=1, model="small")
                if enc:
                    encodings.append(enc[0])
                    names.append(person)
    
    return encodings, names

def main():
    setup_dirs()
    known_encs, known_names = load_encodings_fast()
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    frame_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_count += 1
            if frame_count % PROCESS_EVERY != 0:
                continue
                
            small_frame = cv2.resize(frame, (0, 0), fx=FRAME_SCALE, fy=FRAME_SCALE)
            rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            face_locs = face_recognition.face_locations(rgb_small, number_of_times_to_upsample=1, model="hog")
            
            display_frame = frame.copy()
            
            # Get current time
            current_time = datetime.now().strftime("%H:%M:%S")
            
            for (top, right, bottom, left) in face_locs:
                top *= 2; right *= 2; bottom *= 2; left *= 2
                
                face_enc = face_recognition.face_encodings(
                    rgb_small,
                    known_face_locations=[(top//2, right//2, bottom//2, left//2)],
                    num_jitters=1,
                    model="small"
                )
                
                if face_enc and known_encs:
                    dists = face_recognition.face_distance(known_encs, face_enc[0])
                    best_idx = np.argmin(dists)
                    confidence = (1 - dists[best_idx]) * 100
                    
                    if dists[best_idx] < RECOG_THRESH:
                        name = known_names[best_idx]
                        color = (0, 255, 0)
                        status = f"{name} {confidence:.1f}%"
                    else:
                        status = "Unknown"
                        color = (0, 0, 255)
                    
                    cv2.rectangle(display_frame, (left, top), (right, bottom), color, 2)
                    cv2.putText(display_frame, status, (left + 6, bottom + 25), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
            
            # Display different information on left side
            cv2.putText(display_frame, current_time, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(display_frame, f"Faces: {len(face_locs)}", (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
            cv2.putText(display_frame, "SPACE:Capture | Q:Quit", (10, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
            
            cv2.imshow('Face Recognition', display_frame)
            
            key = cv2.waitKey(1)
            if key in [ord('q'), ord('Q')]:
                break
            elif key == 32 and face_locs:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                img_path = f"known_faces/Amar/Amar_{timestamp}.jpg"
                cv2.imwrite(img_path, frame)

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()